In [28]:
import numpy as np
import pandas as pd

In [55]:
data_raw = pd.read_parquet("datasets/result-1681818213560.parquet.gzip")\
    .drop_duplicates(subset=None, keep='first', inplace=False)
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915706 entries, 0 to 915705
Data columns (total 39 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   vin                       915706 non-null  object 
 1   collect_ts                915706 non-null  int64  
 2   uuid                      915706 non-null  object 
 3   model_name                915706 non-null  object 
 4   series_code               915706 non-null  object 
 5   msg_create_time           915706 non-null  object 
 6   vcu_VehicleMode           0 non-null       float32
 7   vcu_vehicleSpeed          0 non-null       float32
 8   VCU_VehicleSpeedValid     0 non-null       float32
 9   vcu_state                 0 non-null       float32
 10  bms_SOC                   0 non-null       float32
 11  bms_state                 0 non-null       float32
 12  BCM_ebs_U_BATT            103894 non-null  float32
 13  BCM_ebs_SOC               103894 non-null  f

1. ebs_Q_Charge ebs_Q_Discharge BCM_ebs_I_BATT  dcdc_OverTemperature 一直为 0.0，同时又不是状态值，直接删除这些特征
2. IC_LowBatteryvoltage  DCDC_realInputVoltage  DCDC_realInputCurrent 全为空，直接删除这些特征
3. DCDC_MCU_HVIL_Warning DCDC_BMS_HVIL_Warning DCDC_HV_InterlockStatus1 DCDC_HV_InterlockStatus dcdc_deratingStatus 作为状态值先保留
4. BCM_ebs_U_BATT 中间存在 3.0 的值将这部分提取出来进一步观察

In [43]:
data_raw.describe()

,collect_ts,BCM_ebs_U_BATT,BCM_ebs_SOC,BCM_ebs_SOH_SUL,ebs_Q_Charge,ebs_Q_Discharge,BCM_ebs_SOC_STATE,BCM_ebs_SOH_SUL_STATE,BCM_ebs_I_BATT,bcm_12VBatterySOC,...,DCDC_MaxPwrTime,vcu_dcdc_enable,vcu_dcdc_voltageReq,DCDC_MCU_HVIL_Warning,DCDC_BMS_HVIL_Warning,DCDC_HV_InterlockStatus1,DCDC_HV_InterlockStatus,dcdc_systemStatus,dcdc_deratingStatus,dcdc_realTemperature
count,7.774790e+05,87488.000000,87488.000000,87488.000000,87488.0,87488.0,87492.000000,87492.000000,87492.0,91097.000000,...,688461.000000,55712.000000,55712.000000,756198.0,756198.0,756198.0,777479.000000,777479.000000,777479.000000,777479.000000
mean,1.681754e+12,14.049490,81.095787,87.845177,0.0,0.0,1.725392,0.695926,0.0,81.712097,...,33.265335,0.955611,14.086404,0.0,0.0,0.0,0.000078,1.961785,0.000027,37.194752
std,3.395991e+07,0.870883,14.898796,13.562264,0.0,0.0,0.511140,0.461059,0.0,15.029283,...,324.510284,0.205960,0.551321,0.0,0.0,0.0,0.008857,0.233351,0.005197,11.051042
min,1.681552e+12,3.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,10.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-40.000000
25%,1.681721e+12,14.119141,70.000000,81.000000,0.0,0.0,2.000000,0.000000,0.0,70.000000,...,0.000000,1.000000,14.200000,0.0,0.0,0.0,0.000000,2.000000,0.000000,29.000000
50%,1.681749e+12,14.173828,81.000000,89.000000,0.0,0.0,2.000000,1.000000,0.0,81.000000,...,0.000000,1.000000,14.200000,0.0,0.0,0.0,0.000000,2.000000,0.000000,39.000000
75%,1.681788e+12,14.202148,92.000000,100.000000,0.0,0.0,2.000000,1.000000,0.0,98.000000,...,0.000000,1.000000,14.200000,0.0,0.0,0.0,0.000000,2.000000,0.000000,43.000000
max,1.681804e+12,18.999023,255.000000,255.000000,0.0,0.0,3.000000,3.000000,0.0,255.000000,...,4095.000000,1.000000,14.200000,0.0,0.0,0.0,1.000000,6.000000,1.000000,63.000000


In [44]:
drop_columns = ["ebs_Q_Charge", "ebs_Q_Discharge", "BCM_ebs_I_BATT", "dcdc_OverTemperature","IC_LowBatteryvoltage", "DCDC_realInputVoltage", "DCDC_realInputCurrent"]
data_raw = data_raw.drop(columns=drop_columns)
data_raw.describe()

,collect_ts,BCM_ebs_U_BATT,BCM_ebs_SOC,BCM_ebs_SOH_SUL,BCM_ebs_SOC_STATE,BCM_ebs_SOH_SUL_STATE,bcm_12VBatterySOC,dcdc_realOutputVoltage,dcdc_realOutputCurrent,DCDC_AlerState,...,DCDC_MaxPwrTime,vcu_dcdc_enable,vcu_dcdc_voltageReq,DCDC_MCU_HVIL_Warning,DCDC_BMS_HVIL_Warning,DCDC_HV_InterlockStatus1,DCDC_HV_InterlockStatus,dcdc_systemStatus,dcdc_deratingStatus,dcdc_realTemperature
count,7.774790e+05,87488.000000,87488.000000,87488.000000,87492.000000,87492.000000,91097.000000,777479.000000,777479.000000,777479.000000,...,688461.000000,55712.000000,55712.000000,756198.0,756198.0,756198.0,777479.000000,777479.000000,777479.000000,777479.000000
mean,1.681754e+12,14.049490,81.095787,87.845177,1.725392,0.695926,81.712097,13.969805,25.273043,0.001232,...,33.265335,0.955611,14.086404,0.0,0.0,0.0,0.000078,1.961785,0.000027,37.194752
std,3.395991e+07,0.870883,14.898796,13.562264,0.511140,0.461059,15.029283,1.136339,19.397821,0.040110,...,324.510284,0.205960,0.551321,0.0,0.0,0.0,0.008857,0.233351,0.005197,11.051042
min,1.681552e+12,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,10.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-40.000000
25%,1.681721e+12,14.119141,70.000000,81.000000,2.000000,0.000000,70.000000,14.100000,9.300000,0.000000,...,0.000000,1.000000,14.200000,0.0,0.0,0.0,0.000000,2.000000,0.000000,29.000000
50%,1.681749e+12,14.173828,81.000000,89.000000,2.000000,1.000000,81.000000,14.100000,20.200001,0.000000,...,0.000000,1.000000,14.200000,0.0,0.0,0.0,0.000000,2.000000,0.000000,39.000000
75%,1.681788e+12,14.202148,92.000000,100.000000,2.000000,1.000000,98.000000,14.100000,35.900002,0.000000,...,0.000000,1.000000,14.200000,0.0,0.0,0.0,0.000000,2.000000,0.000000,43.000000
max,1.681804e+12,18.999023,255.000000,255.000000,3.000000,3.000000,255.000000,14.200000,149.300003,2.000000,...,4095.000000,1.000000,14.200000,0.0,0.0,0.0,1.000000,6.000000,1.000000,63.000000


In [45]:
pd.DataFrame(data_raw["BCM_ebs_U_BATT"].value_counts()).sort_values(by='BCM_ebs_U_BATT')

,count
BCM_ebs_U_BATT,
3.000000,444
7.016602,1
7.070312,1
7.245117,1
7.551758,1
...,...
14.250977,71
14.251953,39
14.252930,12


In [46]:
data_3 = data_raw[data_raw.BCM_ebs_U_BATT == 3.]
data_3

,vin,collect_ts,uuid,model_name,series_code,msg_create_time,BCM_ebs_U_BATT,BCM_ebs_SOC,BCM_ebs_SOH_SUL,BCM_ebs_SOC_STATE,...,DCDC_MaxPwrTime,vcu_dcdc_enable,vcu_dcdc_voltageReq,DCDC_MCU_HVIL_Warning,DCDC_BMS_HVIL_Warning,DCDC_HV_InterlockStatus1,DCDC_HV_InterlockStatus,dcdc_systemStatus,dcdc_deratingStatus,dcdc_realTemperature
173644,WJF12012541741741,1681715191426,WJF12012541741741#1681715191977#8015,B24DRF6EP2L01,SERES-X1,1681715191446,3.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
173738,WJF12012541741741,1681715201426,WJF12012541741741#1681715201990#1587,B24DRF6EP2L01,SERES-X1,1681715201446,3.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
173812,WJF12012541741741,1681715211427,WJF12012541741741#1681715211996#5750,B24DRF6EP2L01,SERES-X1,1681715211446,3.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
173890,WJF12012541741741,1681715221426,WJF12012541741741#1681715222011#146,B24DRF6EP2L01,SERES-X1,1681715221445,3.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
173988,WJF12012541741741,1681715231426,WJF12012541741741#1681715232035#1727,B24DRF6EP2L01,SERES-X1,1681715231446,3.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210397,WJF12012541741741,1681719581435,WJF12012541741741#1681719582072#7693,B24DRF6EP2L01,SERES-X1,1681719581460,3.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
210518,WJF12012541741741,1681719591435,WJF12012541741741#1681719592038#4688,B24DRF6EP2L01,SERES-X1,1681719591459,3.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
210582,WJF12012541741741,1681719601435,WJF12012541741741#1681719602016#3254,B24DRF6EP2L01,SERES-X1,1681719601472,3.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
210690,WJF12012541741741,1681719611439,WJF12012541741741#1681719612023#9908,B24DRF6EP2L01,SERES-X1,1681719611459,3.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0


In [47]:
# 很明显这部分数据要排除
data_3.describe()

,collect_ts,BCM_ebs_U_BATT,BCM_ebs_SOC,BCM_ebs_SOH_SUL,BCM_ebs_SOC_STATE,BCM_ebs_SOH_SUL_STATE,bcm_12VBatterySOC,dcdc_realOutputVoltage,dcdc_realOutputCurrent,DCDC_AlerState,...,DCDC_MaxPwrTime,vcu_dcdc_enable,vcu_dcdc_voltageReq,DCDC_MCU_HVIL_Warning,DCDC_BMS_HVIL_Warning,DCDC_HV_InterlockStatus1,DCDC_HV_InterlockStatus,dcdc_systemStatus,dcdc_deratingStatus,dcdc_realTemperature
count,4.440000e+02,444.0,444.0,444.0,444.0,444.0,444.0,444.0,444.0,444.0,...,444.0,444.0,444.0,444.0,444.0,444.0,444.0,444.0,444.0,444.0
mean,1.681717e+12,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
std,1.283163e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.681715e+12,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
25%,1.681716e+12,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
50%,1.681717e+12,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
75%,1.681719e+12,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0
max,1.681720e+12,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.0


In [48]:
# 清除异常数据之后的数据
data = data_raw[data_raw.BCM_ebs_U_BATT >= 7.]
data.describe()

,collect_ts,BCM_ebs_U_BATT,BCM_ebs_SOC,BCM_ebs_SOH_SUL,BCM_ebs_SOC_STATE,BCM_ebs_SOH_SUL_STATE,bcm_12VBatterySOC,dcdc_realOutputVoltage,dcdc_realOutputCurrent,DCDC_AlerState,...,DCDC_MaxPwrTime,vcu_dcdc_enable,vcu_dcdc_voltageReq,DCDC_MCU_HVIL_Warning,DCDC_BMS_HVIL_Warning,DCDC_HV_InterlockStatus1,DCDC_HV_InterlockStatus,dcdc_systemStatus,dcdc_deratingStatus,dcdc_realTemperature
count,8.704400e+04,87044.000000,87044.000000,87044.000000,87019.000000,87019.000000,87037.000000,87044.000000,87044.000000,87044.000000,...,80400.000000,5479.000000,5479.000000,87044.0,87044.0,87044.0,87044.0,87044.000000,87044.000000,87044.000000
mean,1.681756e+12,14.105851,81.509445,88.293266,1.734288,0.699571,81.509163,14.014090,22.872591,0.000954,...,152.417725,0.969702,14.133345,0.0,0.0,0.0,0.0,1.969257,0.000023,36.672165
std,3.274048e+07,0.369256,13.761865,12.054450,0.497254,0.459499,13.762928,0.722972,17.155041,0.034051,...,692.943787,0.171420,0.377125,0.0,0.0,0.0,0.0,0.191430,0.004793,9.536016
min,1.681552e+12,7.016602,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,12.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,-40.000000
25%,1.681724e+12,14.121094,70.000000,81.000000,2.000000,0.000000,70.000000,14.100000,9.800000,0.000000,...,0.000000,1.000000,14.200000,0.0,0.0,0.0,0.0,2.000000,0.000000,29.000000
50%,1.681751e+12,14.173828,81.000000,89.000000,2.000000,1.000000,81.000000,14.100000,20.100000,0.000000,...,0.000000,1.000000,14.200000,0.0,0.0,0.0,0.0,2.000000,0.000000,37.000000
75%,1.681788e+12,14.202148,92.000000,100.000000,2.000000,1.000000,92.000000,14.100000,30.799999,0.000000,...,0.000000,1.000000,14.200000,0.0,0.0,0.0,0.0,2.000000,0.000000,41.000000
max,1.681804e+12,18.999023,255.000000,255.000000,3.000000,3.000000,255.000000,14.200000,141.899994,2.000000,...,4095.000000,1.000000,14.200000,0.0,0.0,0.0,0.0,6.000000,1.000000,63.000000


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87044 entries, 6036 to 777476
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   vin                       87044 non-null  object 
 1   collect_ts                87044 non-null  int64  
 2   uuid                      87044 non-null  object 
 3   model_name                87044 non-null  object 
 4   series_code               87044 non-null  object 
 5   msg_create_time           87044 non-null  object 
 6   BCM_ebs_U_BATT            87044 non-null  float32
 7   BCM_ebs_SOC               87044 non-null  float32
 8   BCM_ebs_SOH_SUL           87044 non-null  float32
 9   BCM_ebs_SOC_STATE         87019 non-null  float32
 10  BCM_ebs_SOH_SUL_STATE     87019 non-null  float32
 11  bcm_12VBatterySOC         87037 non-null  float32
 12  dcdc_realOutputVoltage    87044 non-null  float32
 13  dcdc_realOutputCurrent    87044 non-null  float32
 14  DCDC_Al

In [52]:
data[data["BCM_ebs_U_BATT"] < 10.5]["vin"].value_counts()

vin
LM8F7D890MAS00205    49
LM8F7D89XMAS00244    48
LM8F7D897MAS00198    42
LM8F7D795MAS00332     3
LM8F7D898MA008093     2
LM8F7D992MAS00656     1
Name: count, dtype: int64